# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [5]:
import sqlite3
conn = sqlite3.connect('../../assets/datasets/enron.db') 
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[(u'MessageBase',), (u'RecipientBase',), (u'EmployeeBase',)]

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [16]:
Employee_Columns = c.execute("PRAGMA table_info(EmployeeBase)").fetchall()
Employee_Column_Types = [x[2] for x in Employee_Columns]
Employee_Columns = [x[1] for x in Employee_Columns]

Message_Columns = c.execute("PRAGMA table_info(MessageBase)").fetchall()
Message_Column_Types = [x[2] for x in Message_Columns]
Message_Columns = [x[1] for x in Message_Columns]

Recipient_Columns = c.execute("PRAGMA table_info(RecipientBase)").fetchall()
Recipient_Column_Types = [x[2] for x in Recipient_Columns]
Recipient_Columns = [x[1] for x in Recipient_Columns]

print "Employee columns: ", Employee_Columns
print "Employee column types: " , Employee_Column_Types
print "Message columns: " , Message_Columns
print "Message column types: ", Message_Column_Types
print "recipient columns: ", Recipient_Columns
print "recipient column types: ", Recipient_Column_Types

 Employee columns:  [u'eid', u'name', u'department', u'longdepartment', u'title', u'gender', u'seniority']
Employee column types:  [u'INTEGER', u'TEXT', u'TEXT', u'TEXT', u'TEXT', u'TEXT', u'TEXT']
Message columns:  [u'mid', u'filename', u'unix_time', u'subject', u'from_eid']
Message column types:  [u'INTEGER', u'TEXT', u'INTEGER', u'TEXT', u'INTEGER']
recipient columns:  [u'mid', u'rno', u'to_eid']
recipient column types:  [u'INTEGER', u'INTEGER', u'INTEGER']


1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [19]:
print c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
print c.execute("SELECT * FROM MessageBASE LIMIT 5;").fetchall()
print c.execute("SELECT * FROM RecipientBase LIMIT 5;").fetchall()



[(1, u'John Arnold', u'Forestry', u'ENA Gas Financial', u'VP Trading', u'Male', u'Senior'), (2, u'Harry Arora', u'Forestry', u'ENA East Power', u'VP Trading', u'Male', u'Senior'), (3, u'Robert Badeer', u'Forestry', u'ENA West Power', u'Mgr Trading', u'Male', u'Junior'), (4, u'Susan Bailey', u'Legal', u'ENA Legal', u'Specialist Legal', u'Female', u'Junior'), (5, u'Eric Bass', u'Forestry', u'ENA Gas Texas', u'Trader', u'Male', u'Junior')]
[(1, u'taylor-m/sent/11', 910930020, u'Cd$ CME letter', 138), (2, u'taylor-m/sent/17', 911459940, u'Indemnification', 138), (3, u'taylor-m/sent/18', 911463840, u'Re: Indemnification', 138), (4, u'taylor-m/sent/23', 911874180, u'Re: Coral Energy, L.P.', 138), (5, u'taylor-m/sent/27', 912396120, u'Bankruptcy Code revisions', 138)]
[(1, 1, 59), (2, 1, 15), (3, 1, 15), (4, 1, 109), (4, 2, 49)]


Import each of the 3 tables to a Pandas Dataframes

In [66]:
import pandas as pd

df_employee_base = pd.DataFrame(c.execute("SELECT * FROM EmployeeBase").fetchall(), columns = Employee_Columns)

df_message_base = pd.DataFrame(c.execute("SELECT * FROM MessageBase").fetchall(), columns = Message_Columns)

df_recipient_base = pd.DataFrame(c.execute("SELECT * FROM RecipientBase").fetchall(), columns = Recipient_Columns)

df_employee_base.head()

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [23]:
# 1. How many employees are there in the company?
print len(df_employee_base)
#2. How many messages are there in the database?
print len(df_message_base)

156
21635


In [26]:
# Convert the timestamp column in the messages.
df_message_base['Date_Time'] = pd.to_datetime(df_message_base['unix_time'],unit='s')
# When was the oldest message sent? 
print min(df_message_base['Date_Time'])
#And the newest?
print max(df_message_base['Date_Time'])

1998-11-13 04:07:00
2002-06-21 13:37:34


In [86]:
# Some messages are sent to more than one recipient.
# Group the messages by message_id and count the number of recepients. 
df_merged = df_message_base.merge(df_recipient_base, how = 'inner', on = 'mid')
pivot = pd.pivot_table(df_merged, values = ['rno'], index = ['mid'], aggfunc = len)
pivot
# How many messages have only one recepient?
print len(pivot[pivot.rno == 1])
# How many messages have >= 5 recepients?
print len(pivot[pivot.rno >= 5])
# What's the highest number of recepients?
print max(pivot.rno)
# Who sent the message with the highest number of recepients
print pivot[pivot.rno == max(pivot.rno)]
df_merged_2 = df_merged.merge(df_employee_base, how = 'inner', left_on = 'from_eid', right_on = 'eid')
print df_merged_2.ix[df_merged_2.mid == 12116][['name']][0:1]

14985
1380
57
       rno
mid       
12116   57
12151   57
                   name
21839  John J. Lavorato


Rescale to investigate the tail of the curve

In [87]:
from bokeh.charts import Histogram, show, output_notebook

x = Histogram(df_merged, values = 'rno')
show(x)

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

In [176]:
df_merged_2 = df_merged.merge(df_employee_base, how = 'inner', left_on = 'from_eid', right_on = 'eid')

# Are there more Men or Women employees?
print "Males: ", len(df_employee_base[df_employee_base.gender == 'Male'])
print "Females: ", len(df_employee_base[df_employee_base.gender == 'Female'])

print "\n"

# How is gender distributed across departments?
print "Gender Distribution"
print pd.pivot_table(df_employee_base[['gender','department']], columns = ['gender'], index = ['department'], aggfunc = len)

print "\n"

# Who is sending more emails? Men or Women?
print 'Male Emails: ', len(df_merged_2[df_merged_2.gender == 'Male'])
print 'Female Emails: ', len(df_merged_2[df_merged_2.gender == 'Female'])

print "\n"

#What's the average number of emails sent by each gender?

print 'Average Emails By Gender'
males = len(df_merged_2[df_merged_2.gender == 'Male']['mid'].value_counts()) / len(df_employee_base[df_employee_base.gender == 'Male'])
females = len(df_merged_2[df_merged_2.gender == 'Male']['mid'].value_counts()) / len(df_employee_base[df_employee_base.gender == 'Female'])

print 'Males: ', males
print 'Females: ', females

print "\n"

# Are there more Juniors or Seniors?
print 'Are there more Juniors or Seniors?', df_merged_2.groupby('seniority')['eid'].nunique()
print "\n"
print 'Who is sending more emails? Juniors or Seniors?', df_merged_2.groupby('seniority')['mid'].nunique()

print "\n"

print 'Which department is sending more emails? How does that relate with the number of employees in the department? ',
print pd.pivot_table(df_merged_2, values = ['mid','eid'], index = ['department'], aggfunc = {'mid': len, 'eid': lambda x: len(x.unique())})
print "\n"
print 'Who are the top 3 senders of emails? (people who sent out the most emails)'
print df_merged_2['name'].value_counts()[0:3]



Males:  113
Females:  43


Gender Distribution
gender      Female  Male
department              
Forestry        10    50
Legal           13    12
Other           20    51


Male Emails:  22798
Female Emails:  15590


Average Emails By Gender
Males:  113
Females:  298


Are there more Juniors or Seniors? seniority
Junior    78
Senior    74
Name: eid, dtype: int64


Who is sending more emails? Juniors or Seniors? seniority
Junior     9196
Senior    12439
Name: mid, dtype: int64


Which department is sending more emails? How does that relate with the number of employees in the department?              eid    mid
department            
Forestry     59   7112
Legal        25  19292
Other        68  11984


Who are the top 3 senders of emails? (people who sent out the most emails)
Jeff Dasovich       3844
Tana Jones          2858
James D. Steffes    1666
Name: name, dtype: int64


Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

In [174]:
df_merged_3 = df_recipient_base.merge(df_employee_base, how = 'inner', left_on = 'to_eid', right_on = 'eid')

print 'Who is receiving more emails? Men or Women?'
print df_merged_3.groupby('gender')['mid'].nunique()
print "\n"
print 'Who is receiving more emails? Juniors or Seniors'
print df_merged_3.groupby('seniority')['mid'].nunique()
print "\n"
print 'Which department is receiving more emails? How does that relate with the number of employees in the department?'
print pd.pivot_table(df_merged_3, values = ['mid','eid'], index = ['department'], aggfunc = {'mid': len, 'eid': lambda x: len(x.unique())})
print "\n"
print 'Who are the top 5 receivers of emails?'
print df_merged_3['name'].value_counts()[0:5]




Who is receiving more emails? Men or Women?
gender
Female     9617
Male      15446
Name: mid, dtype: int64


Who is receiving more emails? Juniors or Seniors
seniority
Junior     9966
Senior    14556
Name: mid, dtype: int64


Which department is receiving more emails? How does that relate with the number of employees in the department?
            eid    mid
department            
Forestry     60   8424
Legal        25  16311
Other        69  13653


Who are the top 5 receivers of emails?
James D. Steffes    1797
Richard Shapiro     1730
Mark E. Taylor      1477
Steven J. Kean      1290
Sara Shackleton     1173
Name: name, dtype: int64


Which employees sent the most 'mass' emails?

In [182]:
df_merged_3['mid'].value_counts()

print df_merged_2['name'].value_counts()[0:1]

Jeff Dasovich    3844
Name: name, dtype: int64


Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.